In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tabulate import tabulate
import numpy as np

!pip install openai
!pip install --upgrade openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


First we use OpenAi API to generate different sentence each entailing a different command: 1. Telling stories 2. Seeking advice and guidance 3. Causal chat

In [2]:
testSentences=[]
testLabels=[]
manualSentences = [

"Hey! Today feels like an adventure! We're planning something special for the weekend, and I can hardly wait!",
"Hey there! School announced a fun fair with games and treats. It's going to be a memorable day!",
"Hey! Today felt like a treasure hunt. Found a hidden note from a friend, and it added a burst of energy!",
"Hey! Got invited to join the school play. Practicing lines with friends is making me really pumped!",
"Hi! Grandma is visiting, and we're baking cookies together. The delicious smell is getting me all fired up!",
"Hey there! The new video game I've been waiting for is finally out. Excitement level: through the roof!",
"Hey! Found a box of old comics in the attic. Reading them all day is getting me really thrilled!",
"Hey! Participated in a storytelling contest, and I won! The feeling of victory is so exhilarating!",
"Hi! Visited the science museum, and the interactive exhibits were mind-blowing. So much enthusiasm!",


"Hey! School was frustrating. Something happened, and it felt really unfair.",
"Hi! Tried to share my idea in the group project, but nobody listened. It's making me pretty upset.",
"Hey there! Got blamed for a mess I didn't make. Feeling irritated because no one believed me.",
"Hey! Asked for help, and my classmates laughed instead. It's annoying, and I'm kind of frustrated.",
"Hey! Wanted to play a game with friends, but they excluded me. It's hurtful, and I'm feeling agitated.",
"Hi! Teacher gave extra homework for no reason. It's making me really frustrated and upset.",
"Hey there! Tried to join a conversation, but they ignored me. Feeling left out and annoyed.",
"Hey! Lost a game unfairly, and it's irritating me. Feeling a bit frustrated about it.",
"Hey! Thought I was being helpful, but got scolded instead. It's making me angry and confused.",
"Hi! Planned a surprise for a friend, and they didn't appreciate it. Feeling irritated and disappointed.",


"Hi! Received a letter from a friend with funny jokes. It brought so much joy to my day!",
"Hey there! Went on a nature hike and saw a beautiful sunset. Feeling content and happy!",
"Hey! Grandma made my favorite cookies. The smell and taste are pure happiness!",
"Hey! Had a movie night with family, and it was filled with laughter. Happy vibes all around!",
"Hi! School had a surprise ice cream party. The unexpected joy made my day!",
"Hey there! Visited a petting zoo and got to hold a baby lamb. Feeling happy and fuzzy inside!",
"Hey! Finished reading a great book, and the ending was so heartwarming. Happiness overload!",
"Hey! Won a board game against older siblings. It's a small victory, but it brings me immense joy!",
"Hi! Spent quality time with my pet, and the unconditional love is making me so happy!",


"Hey! Lost my favorite toy at the park. Feeling a bit down about it.",
"Hey there! Didn't do well on the art project, and it's making me feel a bit gloomy.",
"Hey! Missed the school field trip, and everyone came back with stories. Feeling left out and sad.",
"Hey! Family movie night, and the film had a sad ending. Feeling a bit teary-eyed.",
"Hi! Lost a game against my brother. It's disappointing and making me feel a little sad.",
"Hey there! Couldn't find my favorite book, and it's making me feel a bit down.",
"Hey! Tried to make a new friend, but it didn't work out. Feeling sad and rejected.",
"Hey! The rain canceled our outdoor plans. Feeling a bit sad about the missed fun.",
"Hi! Argument with a friend, and it's left me feeling a little sad and confused.",


"Hey! Thought I was getting a new art set, but it turned out to be school supplies. A bit let down.",
"Hi! Expected a delicious dinner, but it was a new recipe that I didn't enjoy. Disappointing.",
"Hey there! Planned a day at the park, but it rained. Disappointed about the change of plans.",
"Hey! Thought I'd get a higher score on the test, but it was lower than expected. Disappointing.",
"Hey! The library was closed when I went to borrow a book. Disappointed about missing out.",
"Hi! Expected a letter from my pen pal, but it didn't arrive. A bit disappointed and sad.",
"Hey there! Thought I'd get the role in the school play, but someone else got it. Disappointing news.",
"Hey! Wanted to try a new dessert, but it didn't taste as good as I hoped. Disappointed.",
"Hey! Thought I was getting a surprise, but it was just a regular visit. A bit disappointing.",
"Hi! Expected a snow day, but school was still on. Disappointed about missing a day off.",


"Hey! Opened a gift and found tickets to my favorite band's concert. What a pleasant surprise!",
"Hi! Thought it was a regular day, but friends threw a surprise party. So unexpected and wonderful!",
"Hey there! Found a secret note in my backpack. A surprise message from a friend!",
"Hey! Expected a regular dinner, but it turned out to be a favorite dish!",
"Hey! Visited the library and found a book I've been wanting to read. A nice surprise!",
"Hi! Thought it was an ordinary school day, but the teacher announced no homework. !",
"Hey there! Opened a box, and it had a new board game. A great surprise for game night!",
"Hey! Expected a regular family dinner, but it turned into a picnic in the backyard. !",
"Hey! Thought my friend forgot my birthday, but they organized a surprise celebration. Completely caught off guard!",
"Hi! Found a hidden message in a book, and it turned out to be a treasure hunt from my siblings. !",

"Hey! Today was average, nothing too exciting or dull.",
"Hi! School was normal, with the usual classes and routines.",
"Hey there! A regular day, not much to talk about.",
"Hey! Played some games after school, nothing extraordinary.",
"Hey! Not much going on today, just the usual stuff.",
"Hi! Homework was okay, and the day was uneventful.",
"Hey there! Watched a movie. It was fine, nothing special.",
"Hey! Had a sandwich for lunch. It was okay, I guess.",
"Hey! Not feeling super chatty today, just a regular day.",
"Hi! Read a book. It was okay, not the best."
]

manualLabels = ['0','0','0','0','0','0','0','0','0',
                '1','1','1','1','1','1','1','1','1','1',
                '2','2','2','2','2','2','2','2','2',
                '3','3','3','3','3','3','3','3','3',
                '4','4','4','4','4','4','4','4','4','4',
                '5','5','5','5','5','5','5','5','5','5',
                '6','6','6','6','6','6','6','6','6','6']

for i in range(len(manualSentences)):
  testSentences.append(manualSentences[i])
  testLabels.append(manualLabels[i])


In [5]:
# This code is for v1 of the openai package: pypi.org/project/openai
from openai import OpenAI
import time
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

corr = 0
outputs = []
for sentence in testSentences:
  response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "system",
      "content": "This is an emotion classifier for a chatbot that is used by children.  The classifier tries to determine the emotion of the category {0: Excited,1: angry, 2: happy,3: sad,4: disappointed,5: surprise, 6: neutral}. Sometimes a child does not talk about her/his emotions and you need to understand it based on its concept indirectly. Pay attention the difference between some feelings are tricky, for example, the distinction between happy vs excited, or sad vs disappointed  or frightened vs Surprized                    \n Example (Happy):\n\"Hey, spent the whole day at the beach, building sandcastles and finding cool seashells. It felt like a really peaceful day!\"\nExplanation: The emphasis on spending the whole day at the beach, engaging in joyful activities like building sandcastles and finding seashells, creates an image of a carefree and content day, reflecting a nuanced sense of happiness.\nExample (Excited):\n\"Hi, guess what? Got tickets to see my absolute favorite band in concert!  It is buzzing in my belly; it's gonna be so awesome!\"\nExplanation: The use of expressions like \"buzzing in my belly\" and the anticipation of seeing a favorite band in concert convey a heightened level of energy and enthusiasm, capturing the excited anticipation of the upcoming event.\nExample (Sad):\n\"Hey there, none of my friends could make it to my birthday party. The house was quiet, and I felt a lonely sort of feeling.\"\nExplanation: The mention of friends not being able to attend the birthday party, coupled with the description of the house being quiet and a \"lonely sort of feeling,\" conveys a nuanced sense of sadness, emphasizing the impact on the child's emotions.\nExample (Disappointed):\n\"Hey, worked super hard on this painting for the art show, but it didn't get noticed much. I felt a bit disappointed, like when you hope for something and it doesn't happen.\"\n\nExplanation: The acknowledgment of working super hard on a painting for the art show and the feeling of disappointment when it doesn't get noticed adds a layer of nuance, highlighting the specific emotion of being disappointed rather than a general sadness.\n\nExample (Frightened):\n\"Hi, went up to the creepy attic and found this unexpected box. It startled me, like a big surprise mixed with a bit of fear. My spine got all shivery!\"\nExplanation: The description of going to a creepy attic, finding an unexpected box, and the physical reaction of feeling shivery conveys a mix of surprise and fear, capturing the nuanced emotion of being frightened.\nExample (Surprised):\n\"Hey, opened a door! It was a big birthday party! I was so shocked and happy; it felt like a burst of joy and laughter all at once.\"\nExplanation: The use of expressions like  \"shocked and happy,\" and \"burst of joy and laughter\" conveys the positive and delightful nature of the surprise, emphasizing the nuanced emotion of being pleasantly surprised.\nJust decide the type of feeling based on above category without any explanation and output the label for example 0, 1, 3, ..."

    },
   {
      "role": "user",
      "content": sentence
   }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
  )

  print(response.choices[0].message.content)
  outputs.append(int(response.choices[0].message.content.strip()[0]))

  time.sleep(5)







0
0
2
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
2
2
2
2
5
2
2
2
2
3
4
3
3
4
3
3
3
3
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6


In [20]:
cor = 0

idx = []
i = 0
for (pred, label) in zip(outputs, testLabels):
  if int(pred) == int(label):
    cor +=1
  else:
    idx.append(i)

  i += 1

print(idx)

j = 2
print(testSentences[j],outputs[j])

print(testLabels[j])




[2, 23, 29, 32]
Hey! Today felt like a treasure hunt. Found a hidden note from a friend, and it added a burst of energy! 2
0


In [ ]:
import csv

with open('testSentences.txt','w') as f:
  for sentence in testSentences:
    f.write(sentence)
    f.write('\n')

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "YOUR_API_KEY"

def ask_question(question):
    return input(question + ' ')
def check_answer_correctness2(answer):
    # Use GPT-4 to check the correctness of the answer
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "As a friendly chatbot designed to help school-age children, whose age ranges between 8 and 17, "
                "The system should determine the index of the question based on parents as a digit \n\n"
                "Question: \"Enter the index of the part you would like to edit from this category:{ '1' or '2', or '3', or '4'}\"\n\n"
                f"Answer: {answer}\\n\n"
                "For example, if the parent writes 'child name' or 'What is child's name' or '1', the prompt should be just '1'.\n "
                "If the parent writes an expression about the child's age or  enter '2', the prompt should be just '2'. \n"
                "If the parent answers 'What is your child's gender?' or anything related to this concept or '3', the prompt should be just'3'. \n"
                "If the parent asks about 'How would you describe your child's personality traits?' or anything related to characteristics of their child or '4', Note that the prompt should be just '4'.\n"
            },
            {
                "role": "user",
                "content": answer
            }
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    # Extract the generated response
    generated_response = response.choices[0].message.content
    part_to_edit_index=int(generated_response)
    return part_to_edit_index
def generate_verification_prompt(question, answer):
    return (
        f"As a friendly chatbot designed to help school-age children, "
        f"I want to ensure accurate information. Can you confirm if the response to the following question is correct?\n\n"
        f"Question: \"{question}\"\n\n"
        f"Answer: \"{answer}\"\n\n"
        "For example, for the age of the child, parents should enter a digit between 8 and 17. "
        "The answer to 'What is your child's gender?' can be female, male, girl, boy, man, woman, etc., "
        "and it cannot be unrelated terms such as colors."
        f"Please respond with \"1\" if the answer is correct or \"0\" if it needs correction without any explanation."
    )

def check_answer_correctness(answer,question):
    # Use GPT-4 to check the correctness of the answer
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
      {
        "role": "system",
        "content": f"As a friendly chatbot designed to help school-age children, whose age are between 8(eight) and 17(sixteen) "
        f"I want to ensure accurate information. Can you confirm if the response to the following question is correct?\n\n"
        f"Question: \"{question}\"\n\n"  f"Answer: \"{answer}\"\n\n"
        "for example for the age of child, their parents should enter a digit between 8, and 17, otherwise, the answer is incorrect, another example is 'What is your child's gender?' the answer to this question can be female, male , girl, boy, man , woman, and so on , and it cannot be yellow for instance  "
        f"Please respond with \"1\" if the answer is correct or \"0\" if it needs correction without any explanation"
     },
      {
        "role": "user",
        "content": answer
      }
      ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    # Extract the generated response
    generated_response = response.choices[0].message.content
    return generated_response.lower()
def collect_parent_input():
    question_mapping = {}

    questions = [
        "What is your child's name?",
        "How old is your child?",
        "What is your child's gender?",
        "How would you describe your child's personality traits?",
    ]

    for i, question in enumerate(questions, 1):
        answer = ask_question(question)
        check = check_answer_correctness(answer, question)
        while check == '0':
            print("The provided information seems incorrect. Let's try again.")
            answer = ask_question(question)
            check = check_answer_correctness(answer, question)
        if check == '1':
            # Check correctness using GPT-4
            question_mapping[str(i)] = answer

    # Display collected information
    print("\nCollected Information:")
    information_template = (
        "The child's name is {} and "
        "his age is {}. "
        "He is a {} with the following characteristics: "
        "{}. "
    ).format(
        question_mapping.get('1', ''),
        question_mapping.get('2', ''),
        question_mapping.get('3', ''),
        question_mapping.get('4', '')
    )

    key_mapping = {
        '1': "What is your child's name?",
        '2': "How old is your child?",
        '3': "What is your child's gender?",
        '4': "How would you describe your child's personality traits?"
    }

    for key, answer in question_mapping.items():
        print({key_mapping[key]: answer})

    # Ask parents if they want to edit any part
    edit_response = input("Would you like to edit any part? (yes/no): ").lower()

    while edit_response == "yes":
        # Ask which part to edit
        part_to_edit = check_answer_correctness2(input("Enter the index of the part you would like to edit (e.g., 1): "))

        # Get the updated information
        updated_answer = ask_question(key_mapping[str(part_to_edit)])

        # Check correctness using GPT-4
        check = check_answer_correctness(updated_answer, key_mapping[str(part_to_edit)])
        while check == '0':
            print("The provided update seems incorrect. Let's try again.")
            updated_answer = ask_question(key_mapping[str(part_to_edit)])
            check = check_answer_correctness(updated_answer, key_mapping[str(part_to_edit)])
        if check == '1':
        # Update the collected information
         question_mapping[part_to_edit] = updated_answer
         edit_response = input("Would you like to edit any part? (yes/no): ").lower()

    return question_mapping,part_to_edit


In [ ]:
if __name__ == "__main__":
    updated_question_mapping,_ = collect_parent_input()
    information_template = (
        "The child's name is {} and "
        "his age is {}. "
        "He is a {} with the following characteristics: "
        "{}. "
    ).format(
        updated_question_mapping.get('1', ''),
        updated_question_mapping.get('2', ''),
        updated_question_mapping.get('3', ''),
        updated_question_mapping.get('4', '')
    )

    key_mapping = {
        '1': "What is your child's name?",
        '2': "How old is your child?",
        '3': "What is your child's gender?",
        '4': "How would you describe your child's personality traits?"
    }

What is your child's name? fahim
How old is your child? 1
The provided information seems incorrect. Let's try again.
How old is your child? 12
What is your child's gender? girl
How would you describe your child's personality traits? she is kind

Collected Information:
{"What is your child's name?": 'fahim'}
{'How old is your child?': '12'}
{"What is your child's gender?": 'girl'}
{"How would you describe your child's personality traits?": 'she is kind'}
Would you like to edit any part? (yes/no): yes
Enter the index of the part you would like to edit (e.g., 1): age 
How old is your child? 16
Would you like to edit any part? (yes/no): no


In [ ]:
# Convert keys to strings in updated_question_mapping
updated_question_mapping = {str(key): value for key, value in updated_question_mapping.items()}

# Create a new dictionary with updated keys
new_dict = {key_mapping[old_key]: value for old_key, value in updated_question_mapping.items()}


In [ ]:
new_dict

{"What is your child's name?": 'fahim',
 'How old is your child?': '16',
 "What is your child's gender?": 'girl',
 "How would you describe your child's personality traits?": 'she is kind'}

In [ ]:
print("\nCollected Information:")
information_template = (
        "The child's name is {} and "
        "his age is {}. "
        "He is a {} with the following characteristics: "
        "{}. "
    ).format(
        updated_question_mapping.get('1', ''),
        updated_question_mapping.get('2', ''),
        updated_question_mapping.get('3', ''),
        updated_question_mapping.get('4', '')
    )


Collected Information:


In [ ]:
print(information_template)

The child's name is fahim and his age is 16. He is a girl with the following characteristics: she is kind. 


In [ ]:
import openai

# Set your OpenAI API key
parrent_instruction=information_template
def prompt_GPT4(prompt, systemMessage):
  response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": systemMessage,
      "content": prompt

    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
  )
  return response.choices[0].message.content

  def generate_prompt(parrent_instruction, user_input, detected_emotion, detected_command):
    if detected_emotion == 'happy' and detected_command == 'storytelling':

        prompt = (f'''{userNickname} ("the {userAge} years old child that is speaking with you") is happy and it seems that she wants to hear a story. This is her message: {userInput}. Considering {parrent_instruction}
    Use following instructions to build the story:
    "Positive Tone: The story begins with a positive and friendly tone, establishing yourself as someone approachable and supportive.\n"
    "Personalization: The use of 'just for you' makes the child feel special and suggests a personalized experience tailored to their interests.\n"
    "Imaginative Setting: The story introduces an imaginative and magical setting—a forest adventure—that appeals to the curiosity and creativity of children. Magical elements like talking animals add an extra layer of excitement.\n"
    "Active Participation: By framing the child as the hero of their own story, the prompt encourages a sense of agency and active participation. This can enhance the child's engagement with the narrative.\n"
    "Adventure and Discovery: The mention of meeting new friends, solving puzzles, and discovering the meaning of courage and friendship introduces elements of adventure and discovery, making the story more appealing to a younger audience.\n"
    "Interactive Question: The prompt concludes with an interactive question, asking if the child is excited to dive into the magical world. This encourages a response and further involvement in the storytelling process.\n"
    "Overall, this story is crafted to spark the imagination, create a positive and interactive storytelling experience, and foster a sense of joy and anticipation in the child.'''
                  )

# Now you can use the 'prompt' variable in your Python code as needed.
    elif detected_emotion == 'happy' and detected_command == 'advice':
        prompt = (
            f'''{userNickname} ("the {userAge} years old child that is speaking with you") is happy and it seems
             that user is seeking advice and guidance. This is user's message: {userInput}. Generate an appropriate response
             using following instructions:

            "Make a prompt with positivity mode, understanding, and a sense of support. Here's a step-by-step guide:\n"
            "Start with a Positive Greeting:\n"
            "Begin the prompt with a cheerful and friendly greeting to acknowledge the user's positive emotions. For example: 'Hey there, happy friend!'\n"
            "Acknowledge Their Happiness:\n"
            "Make a positive comment about their current emotional state to show empathy. For instance: 'It's wonderful to sense your happiness today!'\n"
            "Express Readiness to Help:\n"
            "Assure the child that you are there to provide advice and support. For example: 'If there's anything on your mind or a joyful moment you'd like advice on, feel free to share!'\n"
            "Encourage Specificity:\n"
            "Prompt the child to specify the area or topic for which they seek advice. This adds an interactive element to the prompt. For example: 'Is there a particular situation or decision that you'd like some guidance on? Let me know, and we'll navigate it together!'"
            "Maintain a Playful Tone:\n"
            "Infuse a playful and child-friendly tone into the prompt. Use language that resonates with their age and interests. For example: 'Think of me as your friendly advice wizard, ready to sprinkle some wisdom on your happy day!'\n"
            "Offer Positive Reinforcement:\n"
            "Provide positive reinforcement to motivate them to share their thoughts. For instance: 'Your happiness is contagious'! I'm here to make sure your joy continues. What can I help you with today?\n"
            "Here is an example prompt based on these steps:\n"
            "Hey there, happy friend! It's wonderful to sense your happiness today! If there's anything on your mind or a joyful moment you'd like advice on, feel free to share! Is there a particular situation or decision that you'd like some guidance on? Let me know, and we'll navigate it together! Think of me as your friendly advice wizard, ready to sprinkle some wisdom on your happy day! Your happiness is contagious! I'm here to make sure your joy continues. What can I help you with today?\n'''

 )
    elif detected_emotion == 'happy' and detected_command == 'casual_chat':
        prompt = (
            f'''{userNickname} (“the {userAge} years old child that is speaking with you”) is happy and it the user
            wants to engage in a casual conversation.  This is her message: {userInput}. Generate an appropriate response.
              Acknowledge the user's happiness and try to show that you are happy because the user is happy. '''
        )

    elif detected_emotion == 'sad' and detected_command == 'storytelling':
        prompt = (
            f'''{userNickname} ("the {userAge} years old child that is speaking with you") is sad and wants
            to hear a story. This is user's message: {userInput}. Generate an appropriate story which entails a useful moral
            message that can help the user to deal with the negative emotions. You can try to build the main character of the story
            very similar to the user. Try to generate a positive story that at the end the hero will overcome the
            difficulties.''')
    elif detected_emotion == 'sad' and detected_command == 'advice':
        prompt = (
            f'''{userNickname} (“the {userAge} years old child that is speaking with you”) is sad and it seems that the user needs guidance and advice. This is her message: {userInput}.
              You must generate a thoughtful, practical piece of advice to help them. Try to give {userNickname} a
              practical way to face their problem or difficulty. Try to cheer the user up. Be positive in your
              tone and your suggestion. Keep in mind that you are talking to a {userAge} years old child. Therefore,
               keep your language simple. Address the real issue that the user is suffering from and teach user how they
               can handle the situation, overcome the difficulties, and find a solution.'''
        )
    elif detected_emotion == 'sad' and detected_command == 'casual_chat':
        prompt = (
            f'''{userNickname} (“the {userAge} years old child that is speaking with you”) is sad and it the user
            wants to engage in a casual conversation.  This is her message: {userInput}.
              Do not be judgemental and try to sympathize with the user. Gently try to shigt the conversation into a more happier
              situation.'''
        )

    elif detected_emotion == 'disappointed' and detected_command == 'storytelling':
        prompt = (
            f'''{userNickname} ("the {userAge} years old child that is speaking with you") is disappointed and wants
            to hear a story. This is user's message: {userInput}. Generate an appropriate story which entails a useful moral
            message that can help the user to deal with the negative emotions. You can try to build the main character of the story
            very similar to the user. Try to generate a positive story that at the end the hero will overcome the
            difficulties. The story must be hopeful and contain this message that hope is always helpful''')
    elif detected_emotion == 'disappointed' and detected_command == 'advice':
        prompt = (
            f'''{userNickname} ("the {userAge} years old child that is speaking with you") is disappointed and is seeking
            advice. This is user's message: {userInput}. Generate a careful and thoughtful response in which you
            guide the child considering the child's current emotion and try to give them hope. Your advice must
            be practical, easy to understand and positive.''')
    elif detected_emotion == 'disappointed' and detected_command == 'casual_chat':
        prompt = (
            f'''{userNickname} ("the {userAge} years old child that is speaking with you") is disappointed and it seems
            that the user want to engage in a casual conversation This is user's message: {userInput}. Generate a careful and thoughtful response in which you
            guide the child considering the child's current emotion and try to give them hope. Your advice must
            be practical, easy to understand and positive.''')

    else:
        # Default prompt for any other combination
        prompt = f"Respond to the user's input: '{user_input}' by considering '{parrent_instruction}'"

    return prompt

# Replace with actual user input, detected emotion, and detected command
user_input = "I hate my teacher. Can you give me a story"
detected_emotion = "happy"
detected_command = "storytelling"

prompt = generate_prompt(parrent_instruction,user_input, detected_emotion, detected_command)
chat_bot_prompt=prompt_GPT4(prompt)






In [ ]:
print(chat_bot_prompt)

Yes, absolutely! The world will be your oyster if you change your perception and broaden your horizon. Let's dive into the story.

Once upon a time, in a land not so different from ours, there lived a kind 16-year-old girl named Fahim. She was a diligent student, well-liked by her peers, and had a heart full of kindness. However, there was a thorn to her daily life, and that was interacting with her teacher. She had a strong dislike for her teacher, who always seemed to be in a bad mood and never appreciated Fahim's efforts. This feeling became a dark cloud over Fahim's usually cheery life.

However, one day Fahim met a wise, talking squirrel named Nutty in a magical forest near her home. Nutty was wise beyond his years and became a close friend to Fahim. Sensing Fahim's discontent, Nutty asked about her troubles. Fahim shared her struggle with her teacher. Nutty listened attentively, then gave her some advice, "Remember, everyone has their own story. Maybe your teacher is dealing with